# P1 Edgar Data API Client for REST API

## Prerequisites

* Python version >= 3.7
* jupyter notebook

## Initialization

In [4]:
import p1_data_client_python.edgar_client as p1_edg

# Enter your token here.
# You can get your token by signing up at `www.particle.one`.
#TOKEN = "YOUR_TOKEN_HERE"
# An example token is like:

TEST_BASE_URL = 'http://etl.p1:5001/edgar/v1/'
TOKEN = "1234567890"

client = p1_edg.EdgarClient(token="1234567890", base_url=TEST_BASE_URL)

## Quick start

### Entities description

* [Mappers](#mappers)
* [Payload data](#payload)

So, we have make two simple steps: 

1. Obtain the information about identifiers.
2. Download payload data by certain arguments.


<a id="mappers"></a>
## Mappers

### GvkeyCikMapper

Handle Cik <-> Gvkey transformation.

Take a look at the following examples:

In [5]:
gvkey_mapper = p1_edg.GvkeyCikMapper(token=TOKEN,base_url=TEST_BASE_URL)
gvkey_mapper.get_gvkey_from_cik(cik='0000940800', as_of_date='2007-01-18')

,cik,gvkey,effdate,thrudate
0,0000940800,061411,2007-01-18 00:00:00,2007-03-14 23:59:59
1,0000940800,061411,2007-03-15 00:00:00,2007-06-20 23:59:59
2,0000940800,061411,2007-06-21 00:00:00,2008-03-05 23:59:59
3,0000940800,061411,2008-03-06 00:00:00,2008-05-07 23:59:59
4,0000940800,061411,2008-05-08 00:00:00,2008-09-25 23:59:59
5,0000940800,061411,2006-04-28 00:00:00,2006-08-23 23:59:59
6,0000940800,061411,2006-08-24 00:00:00,2007-01-17 23:59:59
7,0000940800,061411,2008-09-26 00:00:00,2010-04-20 15:59:59
8,0000940800,061411,2010-04-20 16:00:00,2011-08-31 15:59:59
9,0000940800,061411,2011-08-31 16:00:00,2018-03-19 23:59:59


In [6]:
gvkey_mapper.get_cik_from_gvkey(gvkey='061411', as_of_date='2007-01-18')

,cik,gvkey,effdate,thrudate
0,0000940800,061411,2006-04-28 00:00:00,2006-08-23 23:59:59
1,0000940800,061411,2006-08-24 00:00:00,2007-01-17 23:59:59


### CompustatItemMapper

Provide mapping between keywords and description of Compustat items.

Here is examples of usage:

In [7]:
item_mapper = p1_edg.CompustatItemMapper(token=TOKEN,
                                         base_url=TEST_BASE_URL)
item_mapper.get_item_from_keywords(
            keywords=['short-term', 'short term'])

,item,description
0,IVSTCHY,Short-Term Investments - Change
1,CHEQ,Cash and Short-Term Investments
2,ALTOQ,Other Long-term Assets
3,IVLTQ,Total Long-term Investments
4,DLTTQ,Long-Term Debt - Total
5,DLTRY,Long-Term Debt - Reduction
6,DLTISY,Long-Term Debt - Issuance


In [8]:
item_mapper.get_mapping()

,item,description
0,ACCHGQ,Accounting Changes / Cumulative Effect
1,ACOMINCQ,Accumulated Other Comprehensive Income (Loss)
2,ACOQ,Current Assets - Other - Total
3,ACTQ,Current Assets - Total
4,ADRRQ,ADR Ratio
...,...,...
167,XIDOQ,Extraordinary Items and Discontinued Operations
168,XINTQ,Interest and Related Expense- Total
169,XIQ,Extraordinary Items
170,XRDQ,Research and Development Expense



<a id="payload"></a>
## Payload data

After collecting all necessary identifiers all together
we ready to download payload data.

Let's see how we can do it by next examples:

In [10]:
client.get_payload(form_name='8-K',
                   cik=1002910,
                   start_date='2021-11-04',
                   end_date='2020-11-04',
                   items=['OIBDPQ', 'NIQ']
                   )

,url,cik,filing_date,internal_timestamp,item,table_row_name,extracted_value,period
0,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,NIQ,Net Income,369.00,2020-11-04T00:00:00
1,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,NIQ,Less: Net Income Attributable to Noncontrollin...,2.00,2020-11-04T00:00:00
2,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,NIQ,Net Income Attributable to Ameren Common Share...,367.00,2020-11-04T00:00:00
3,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Total operating revenues,1628.00,2020-11-04T00:00:00
4,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Other operations and maintenance,418.00,2020-11-04T00:00:00
5,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Taxes other than income taxes,128.00,2020-11-04T00:00:00
6,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Total operating expenses,1134.00,2020-11-04T00:00:00
7,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Operating Income,494.00,2020-11-04T00:00:00
8,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,"Other Income, Net",48.00,2020-11-04T00:00:00
9,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Income Before Income Taxes,432.00,2020-11-04T00:00:00


Ok, time to figure out what search criteria we used to:

- form_name: Edgar form short code
- cik: Company Identification Key 
- start_date and end_date: limit our selection by dates
- items: list of items that we want to see in a result set.

Keep in mind: if we specify too broad range of search,
then result could by quite huge.
